In [ ]:
%matplotlib inline
import pathlib as pl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import simple_soil
import flopy.plot.styles as styles

Maximum infiltration

```
(upslope_hortonian + rain + snowmelt - hru_sroffi - hru_sroffp) x (1-hru_percent_imperv)
```

Conversion factors

In [ ]:
in2m = 1.0 / (12.0 * 3.28081)
acre2sqm = 4046.86

In [ ]:
hru_percent_imperv = 0.309456

Forcing data

In [ ]:
data_dir = pl.Path("data/hru_1")

In [ ]:
tag = " 2"

In [ ]:
# rainfall
rain = pd.read_csv(data_dir / "hru_rain.csv", index_col=0)
snow = pd.read_csv(data_dir / "snowmelt.csv", index_col=0)
sroffi = pd.read_csv(data_dir / "hru_sroffi.csv", index_col=0)
sroffp = pd.read_csv(data_dir / "hru_sroffi.csv", index_col=0)
rain["ppt m/day"] = rain[tag] * in2m
rain["net ppt m/day"] = (
    (rain[tag] + snow[tag] - sroffi[tag] - sroffp[tag])
    * (1.0 - hru_percent_imperv)
    * in2m
)
rain = rain.drop(tag, axis=1)
rain.columns

Soil potential evapotranspiration

```
potet - hru_intcpevap - snow_evap - hru_impervevap - dprst_evap_hru
```

In [ ]:
# potential
pet = pd.read_csv(data_dir / "potet.csv", index_col=0)
intcpevap = pd.read_csv(data_dir / "hru_intcpevap.csv", index_col=0)
snowevap = pd.read_csv(data_dir / "snow_evap.csv", index_col=0)
impervevap = pd.read_csv(data_dir / "hru_impervevap.csv", index_col=0)
dprstevap = pd.read_csv(data_dir / "dprst_evap_hru.csv", index_col=0)

In [ ]:
# soil pet
pet["pet m/day"] = (
    pet[tag]
    - intcpevap[tag]
    - snowevap[tag]
    - impervevap[tag]
    - dprstevap[tag]
) * in2m
pet = pet.drop(tag, axis=1)
pet.columns

In [ ]:
forcing_mosaic = """
    AA
    """

In [ ]:
with styles.USGSPlot():
    fig = plt.figure(layout="constrained", figsize=(6, 2))
    ax_dict = fig.subplot_mosaic(forcing_mosaic, sharex=True)

    ax = ax_dict["A"]
    rain.plot(ax=ax, lw=0.5, color="cyan")
    pet.plot(ax=ax, lw=0.5, color="red")

Control volume

In [ ]:
cv = simple_soil.base.ControlVolume(
    thickness=2.0,
    max_vertical_rate=0.0305,
    horizontal_vertical_ratio=0.0,
    theta0=0.3,
    theta_wp=0.1,
    theta_fc=0.3,
    theta_sat=0.44,
    # infiltration_method="green-ampt",
    soil="loam",
)
cv.infiltration_method.set_infiltration_time(1.0)

In [ ]:
time_slice = slice(0,730)
time_slice

In [ ]:
for inflow, pet_rate in zip(rain["ppt m/day"][time_slice], pet["pet m/day"][time_slice]):
    cv.update(inflow_rate=inflow, pet_rate=pet_rate)

In [ ]:
df = cv.get_dataframe(normalize=True)

In [ ]:
df

In [ ]:
mosaic = """
    AB
    CD
    EF
    GH
    """
line_dict = {"lw": 0.5, "color": "black", "clip_on": False}
theta_dict = {"lw": 0.5, "color": "0.5"}

In [ ]:
with styles.USGSPlot():
    fig = plt.figure(layout="constrained", figsize=(9, 6))
    ax_dict = fig.subplot_mosaic(mosaic, sharex=True)

    ax = ax_dict["A"]
    df["theta"].plot(ax=ax, **line_dict)
    ax.axhline(cv.theta_fc, **theta_dict)
    ax.axhline(cv.theta_discharge, **theta_dict)
    ax.axhline(cv.theta_sat, **theta_dict)
    ax.set_ylabel(r"$\theta$")
    
    ax = ax_dict["B"]
    df["rejected_inflow_L/T"].plot(ax=ax, **line_dict)
    ax.set_ylabel("Rejected Inflow")

    ax = ax_dict["C"]
    ax.plot(df.index, rain["ppt m/day"][time_slice], lw=0.5, color="cyan", ls=":")
    df["inflow_L/T"].plot(ax=ax, **line_dict)
    ax.set_ylabel("Inflow")

    ax = ax_dict["D"]
    df["aet_L/T"].plot(ax=ax, **line_dict)
    ax.set_ylabel("Actual\nEvapotranspiration")
        
    ax = ax_dict["E"]
    df["recharge_L/T"].plot(ax=ax, **line_dict)
    ax.set_ylabel("Recharge")
        
    ax = ax_dict["F"]
    df["surface_L/T"].plot(ax=ax, **line_dict)
    ax.set_ylabel("Surface Discharge")

    ax = ax_dict["G"]
    df["lateral_L/T"].plot(ax=ax, **line_dict)
    ax.set_ylabel("Lateral Flow")
    
    ax = ax_dict["H"]
    df["storage_change_L/T"].plot(ax=ax, **line_dict)
    ax.axhline(0.0, **theta_dict)
    ax.set_ylabel("Storage Change")

    fig.align_ylabels()

In [ ]:
inflow_mosaic = """
    AABB
    .CC.
    .CC.
    """
with styles.USGSPlot():
    fig = plt.figure(layout="constrained", figsize=(8, 6))
    ax_dict = fig.subplot_mosaic(inflow_mosaic)

    ax = ax_dict["A"]
    ax.plot(df.index, rain["ppt m/day"][time_slice], lw=0.5, color="cyan")
    df["inflow_L/T"].plot(ax=ax, **line_dict)
    ax.set_ylabel("Inflow")

    ax = ax_dict["B"]
    df["inflow_L/T"].cumsum().plot(ax=ax, **line_dict)
    ax.plot(df.index, rain["ppt m/day"].cumsum()[time_slice], lw=0.5, color="cyan")
    ax.set_ylabel("Cumulative Inflow")

    ax = ax_dict["C"]
    ax.axline((1,1), slope=1, **theta_dict)
    ax.plot(df["inflow_L/T"].cumsum(), rain["ppt m/day"].cumsum()[time_slice], **line_dict)
    ax.set_ylabel("Cumulative Rainfall")
    ax.set_xlabel("Cumulative Inflow")
    axis_max = max(ax.get_xlim()[1], ax.get_ylim()[1])
    ax.set_xlim(0, axis_max)
    ax.set_ylim(0, axis_max)
